In [1]:
# Turning the raw data into a useable format
students = []
output = []
with open("records.csv", "r") as file:
    for row in file:
        students.append(row.strip().split(","))
header = students[0] + ["Team Assigned"]
students = students[1:]
students_split = dict()

for i in students:
    if i[0] in students_split.keys():
        students_split[i[0]].append(i)
    else:
        students_split[i[0]] = [i]

In [3]:
# Visualising data
avg = sum([float(i[5]) for i in students])/6000
for i in set([i[2] for i in students]):
    print( f'{i} (HIGH): {[i[2] for i in students if float(i[5]) >= avg].count(i)}',
           f'(LOW): {[i[2] for i in students if float(i[5]) <= avg].count(i)}',
           f'(Male): {[i[2] for i in students if i[4] == "Male"].count(i)}',
           f'(Female): {[i[2] for i in students if i[4] == "Female"].count(i)}',
           f'{i} (Total): {[i[2] for i in students].count(i)}')
print()
for i in set([i[4] for i in students]):
    print( f'{i} (HIGH): {[i[4] for i in students if float(i[5]) >= avg].count(i)}',
           f'(LOW): {[i[4] for i in students if float(i[5]) <= avg].count(i)}')

for i, j in students_split.items():
    print(f'{i}: {len(set(k[2] for k in j))} Male: {sum(1 for a in j if a[-2] == "Male")} Female: {sum(1 for a in j if a[-2] == "Female")}')

MAE (HIGH): 276 (LOW): 236 (Male): 213 (Female): 299 MAE (Total): 512
SPMS (HIGH): 145 (LOW): 131 (Male): 178 (Female): 98 SPMS (Total): 276
CCDS (HIGH): 387 (LOW): 315 (Male): 359 (Female): 343 CCDS (Total): 702
NIE (HIGH): 101 (LOW): 98 (Male): 130 (Female): 69 NIE (Total): 199
LKCMedicine (HIGH): 42 (LOW): 41 (Male): 39 (Female): 44 LKCMedicine (Total): 83
EEE (HIGH): 412 (LOW): 376 (Male): 425 (Female): 363 EEE (Total): 788
SSS (HIGH): 214 (LOW): 211 (Male): 73 (Female): 352 SSS (Total): 425
ADM (HIGH): 58 (LOW): 58 (Male): 48 (Female): 68 ADM (Total): 116
HASS (HIGH): 1 (LOW): 0 (Male): 1 (Female): 0 HASS (Total): 1
CCEB (HIGH): 244 (LOW): 221 (Male): 115 (Female): 350 CCEB (Total): 465
SBS (HIGH): 164 (LOW): 138 (Male): 98 (Female): 204 SBS (Total): 302
ASE (HIGH): 49 (LOW): 55 (Male): 56 (Female): 48 ASE (Total): 104
CoB (NBS) (HIGH): 418 (LOW): 375 (Male): 302 (Female): 491 CoB (NBS) (Total): 793
SoH (HIGH): 180 (LOW): 152 (Male): 155 (Female): 177 SoH (Total): 332
MSE (HIGH): 

In [ ]:
group_size = int(input("Please choose the group size"))


tutorial_groups = dict.fromkeys(students_split.keys(), [])
for tut_grp, data in students_split.items():

    avg_gpa     = sum([float(i[5]) for i in data]) / 50
    male_high   = sorted([i for i in data if i[4] == "Male"   and float(i[5]) >= avg], key=lambda x: x[5], reverse=True)
    male_low    = sorted([i for i in data if i[4] == "Male"   and float(i[5]) <  avg], key=lambda x: x[5])
    female_high = sorted([i for i in data if i[4] == "Female" and float(i[5]) >= avg], key=lambda x: x[5], reverse=True)
    female_low  = sorted([i for i in data if i[4] == "Female" and float(i[5]) <  avg], key=lambda x: x[5])
    schools     = dict([(j, [i[2] for i in data].count(j)) for j in set([i[2] for i in data])])
 
    project_group = 1
    categories = [male_high, female_high, male_low, female_low]

    grp_count = len(data)//group_size

    for entry in range(grp_count*group_size):

        pos = entry%group_size
        if pos == 0:
            chosen = []
            
        project_group = entry//group_size + 1

        if pos >= group_size - group_size%len(categories):
            pos = [len(i) for i in categories].index(max([len(i) for i in categories]))
        else:
            pos = pos%len(categories)
        category = categories[pos]

        filtered = [i for i in category if i[2] not in chosen]
        if len(filtered) == 0:
            filtered = category
        counts = [schools[i[2]] for i in filtered]
        person = filtered[counts.index(max(counts))]

        schools[person[2]] -= 1
        category.remove(person)
        chosen.append(person[2])

        if len(category) == 0:
            categories.pop(pos)
        else:
            categories[pos] = category       

        output.append(person + [str(project_group)]) 
        tutorial_groups[tut_grp].append(person)
    project_group = 1
    for category in categories:
        for entry in category:
            output.append(entry + [str(project_group)]) 
            project_group += 1

In [22]:
# Checking for outliers

n = 0
avg_gpas = []
for i in range(0, len(output)-group_size, group_size):
    male = sum([1 for j in output[i:i+group_size] if j[4] == "Male"])
    female = sum([1 for j in output[i:i+group_size] if j[4] == "Female"])
    avg_gpas.append(sum([float(j[5]) for j in output[i:i+group_size]])/group_size)

    sch_dist = dict()
    for j in set([i[2] for i in output[i:i+group_size]]):
        count = [i[2] for i in output[i:i+group_size]].count(j)
        sch_dist[j] = count

    if output[i][0] in ["G-114"]:
        print(male, female)
    if male > 4:
        n += 1
        print(output[i][0], male, female, avg)
        print()
    elif female > 4:
        n+= 1
        print(output[i][0], male, female, avg)
        print()
avg_grp = sum(avg_gpas)/1200
sd = (sum((i - avg_grp) for i in avg_gpas)/(1200))**(0.5)
print(sd)
print(n) 

G-1 3 5 4.088658333333333

G-1 2 6 4.088658333333333

G-1 3 5 4.088658333333333

G-10 5 3 4.088658333333333

G-100 1 7 4.088658333333333

G-101 5 3 4.088658333333333

G-101 3 5 4.088658333333333

G-102 2 6 4.088658333333333

G-102 3 5 4.088658333333333

G-104 5 3 4.088658333333333

G-104 1 7 4.088658333333333

G-104 1 7 4.088658333333333

G-105 5 3 4.088658333333333

G-105 3 5 4.088658333333333

G-105 3 5 4.088658333333333

G-106 3 5 4.088658333333333

G-107 0 8 4.088658333333333

G-107 2 6 4.088658333333333

G-109 5 3 4.088658333333333

G-109 2 6 4.088658333333333

G-109 0 8 4.088658333333333

G-11 3 5 4.088658333333333

G-11 3 5 4.088658333333333

G-110 1 7 4.088658333333333

G-110 2 6 4.088658333333333

G-111 5 3 4.088658333333333

G-111 5 3 4.088658333333333

G-112 3 5 4.088658333333333

G-112 2 6 4.088658333333333

G-112 2 6 4.088658333333333

G-113 3 5 4.088658333333333

G-113 2 6 4.088658333333333

4 4
4 4
4 4
2 6
G-114 2 6 4.088658333333333

0 8
G-114 0 8 4.088658333333333

2 6

In [7]:
# Finding out the representative diversity of each tutorial group, by obtaining:
# 1. Average CGPA
# 2. Gender distribution
# 3. School diversity

# This function takes in the list of all students in the tutorial group as a 2D array,
# Returns and prints 1. mean GPA, 2. gender ratio, and 3. number of schools as a dictionary
def analyze_tutorial_groups(data):
    analysis = {}

    # Process each row
    for row in data:
        tutorial_group = row[0]
        gender = row[4]
        cgpa = float(row[5])
        school = row[2]

        # Initialise group statistics if new tutorial group
        if tutorial_group not in analysis:
            analysis[tutorial_group] = {
                'total_cgpa': 0.0,
                'num_students': 0,
                'male_count': 0,
                'female_count': 0,
                'schools': [],
                'students': []
            }

        # Update analysis dictionary for each data row
        analysis[tutorial_group]['total_cgpa'] += cgpa
        analysis[tutorial_group]['num_students'] += 1

        # Append eveery new school to keep track of all unique schools
        if school not in analysis[tutorial_group]['schools']:
            analysis[tutorial_group]['schools'].append(school)

        # Track the gender diversity of tutorial group
        if gender.lower() == 'male':
            analysis[tutorial_group]['male_count'] += 1
        elif gender.lower() == 'female':
            analysis[tutorial_group]['female_count'] += 1

        # Append student information to the students list
        analysis[tutorial_group]['students'].append(row)

    # Calculate and print analysis results for each tutorial group
    for group, stats in analysis.items():
        mean_cgpa = stats['total_cgpa'] / stats['num_students']
        gender_ratio = f"{stats['male_count']}:{stats['female_count']}"
        num_schools = len(stats['schools'])

        # Print these results
        print(f"\nTutorial Group: {group}")
        print(f"Mean CGPA: {mean_cgpa:.2f}")
        print(f"Gender Ratio (Male:Female): {gender_ratio}")
        print(f"Number of Schools: {num_schools}")

    return analysis

In [8]:
for key in grouped_data:
    analyze_tutorial_groups(grouped_data[key])

NameError: name 'grouped_data' is not defined

In [23]:
with open("out.csv", "w") as f:
    f.write(",".join(header) + "\n")
    for i in output:
        f.write(",".join(i) + "\n")